<a href="https://colab.research.google.com/github/SuperNZH/Deep-Learning-Practice/blob/main/Dive%20in%20DL/7.0_byModule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn

# 继承Module

In [3]:
class MLP(nn.Module):
  def __init__(self, **kwargs):
    super(MLP, self).__init__(**kwargs)
    self.hidden = nn.Linear(784, 256)
    self.act = nn.ReLU() # act 是激活函数的意思
    self.output = nn.Linear(256, 10)
  # 定义前向运算，根据x计算返回所需要的模型输出
  def forward(self, x):
    a = self.act(self.hidden(x))
    return self.output(a)

# 不需要定义反向函数，是因为这个class里面系统会自动求梯度，然后自动生成反向传播的backward函数

In [4]:
X = torch.rand(2,784)
print(X)
net = MLP()
print(net)
net(X)

tensor([[0.6184, 0.3753, 0.6375,  ..., 0.8479, 0.2358, 0.0512],
        [0.4935, 0.1910, 0.4643,  ..., 0.6272, 0.8248, 0.0692]])
MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1700, -0.0665, -0.0794, -0.0259, -0.1163, -0.0185, -0.0291, -0.1015,
         -0.1451,  0.1311],
        [ 0.2767, -0.0254, -0.0215, -0.0054, -0.2143,  0.1079, -0.0250, -0.1032,
          0.0278,  0.0299]], grad_fn=<AddmmBackward0>)

# Module子类 --> Sequential类

In [5]:
# 如果模型的前向计算为简单串联各个层的计算时，可以用Sequential来更简单的定义模型
# 这个是Sequential大概的运行逻辑
class MySequential(nn.Module):
  from collections import OrderedDict
  def __init__(self, *args):
    super(MySequential, self).__init__()
    if len(args) == 1 and isinstance(args[0], OrderedDict):
      for key, module in args[0].items():
        self.add_module(key, module)
    else:
      for idx, module in enumerate(args):
        self.add_module(str(idx), module)

  def forward(self, input):
    for module in self._modules.values():
      input = module(input)
    return input

In [6]:
net = MySequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.2311, -0.0846, -0.1451,  0.1224, -0.0441, -0.2211, -0.1441, -0.0295,
          0.1228, -0.2471],
        [ 0.2209, -0.1845, -0.2211, -0.0266, -0.1540, -0.1795, -0.2760,  0.0276,
          0.2003, -0.1763]], grad_fn=<AddmmBackward0>)

# Module子类 --> ModuleList类

In [7]:
# ModuleList 接受一个子模块的列表作为输入，像list一样可以append和extend

net = nn.ModuleList([nn.Linear(784,256), nn.ReLU()])
net.append(nn.Linear(256,10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


# Module子类 --> ModuleDict类

In [8]:
net = nn.ModuleDict({
    'linear': nn.Linear(784,256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256,10)
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


# Module拓展

In [9]:
class FancyMLP(nn.Module):
  def __init__(self, **kwargs):
    super(FancyMLP, self).__init__(**kwargs)
    self.rand_weight = torch.rand((20,20), requires_grad=False)
    self.linear = nn.Linear(20,20)

  def forward(self, x):
    x = self.linear(x)
    x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
    x = self.linear(x)
    while x.norm().item() > 1:
      x /= 2
    if x.norm().item() < 0.8:
      x *= 10
    return x.sum()

In [10]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-19.6512, grad_fn=<SumBackward0>)